In [ ]:
import requests
import nltk
import ipywidgets as widgets
from IPython.display import display

In [ ]:
nltk.download("punkt")  # Download the required NLTK data

API_URL = "https://api-inference.huggingface.co/models/kredor/punctuate-all"
API_TOKEN = "API-TOKEN"  # Replace with your actual API token
headers = {"Authorization": f"Bearer {API_TOKEN}"}


In [ ]:
def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

def punctuate_text(text):
    output = query({
        "inputs": text
    })
    punctuated_text = ""
    for item in output:
        if punctuated_text and item["word"] and not item["word"][0].isspace():
            punctuated_text += " "
        punctuated_text += item["word"]
        if item["entity_group"] in (".", "?", "!"):
            yield punctuated_text.strip()
            punctuated_text = ""


In [ ]:

def handle_text_change(change):
    unpunctuated_text = change["new"].strip()
    output_text.value = ""  # Clear the output text area
    if unpunctuated_text:
        sentences = nltk.sent_tokenize(unpunctuated_text)
        for sentence in sentences:
            # TODO:: if sentence is complete
            punctuated_sentences = punctuate_text(sentence)
            for punctuated_sentence in punctuated_sentences:
                output_text.value += punctuated_sentence + "\n"

In [ ]:
# Create input and output widgets
input_text = widgets.Textarea(placeholder="Enter a sentence", layout=widgets.Layout(width="100%", height="100px"))
output_text = widgets.Textarea(disabled=True, layout=widgets.Layout(width="100%", height="200px"))

# Register the text change event handler
input_text.observe(handle_text_change, names="value")

# Display the widgets
display(input_text, output_text)